# Deep Dive into Langchain

- [Introduction to Langchain](https://python.langchain.com/docs/introduction/)
- [Python Dotenv](https://github.com/theskumar/python-dotenv)

In [1]:
%%capture
!pip install -r requirements.txt -q

In [2]:
import os
import openai
import getpass

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OPENAI_API_KEY: ")
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LANGCHAIN_API_KEY: ")

Enter your OPENAI_API_KEY:  ········
Enter your LANGCHAIN_API_KEY:  ········


In [4]:
# print(os.getenv("OPENAI_API_KEY"))
# print(os.getenv("LANGCHAIN_API_KEY"))

## Langchain Components
### Chat Models: GPT-3.5 Turbo and GPT-4

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
output = llm.invoke("Explain quantum mechanics in one sentence.")
print(output.content)

Quantum mechanics is a branch of physics that describes the behavior of subatomic particles in terms of probabilities and wave functions.


In [6]:
# help(ChatOpenAI)

In [7]:
# using model gpt-4o-mini. Default model is gpt-3.5-turbo
output = llm.invoke("Explain quantum mechanics in one sentence.", model="gpt-4o-mini")
print(output.content)

Quantum mechanics is a fundamental branch of physics that describes the behavior of matter and energy at the smallest scales, where particles exhibit wave-particle duality, uncertainty, and entanglement, challenging classical intuitions about how the universe operates.


#### Langchain and OpenAI Chat Completions API
- [OpenAI Text Generations](https://platform.openai.com/docs/guides/text-generation?lang=python)
- [Langchain openai](https://python.langchain.com/api_reference/openai/index.html)
- [Langchain openai reference](https://python.langchain.com/api_reference/)

In [8]:
from langchain.schema import (
    SystemMessage,
    AIMessage,
    HumanMessage
)

messages = [
    SystemMessage(content="You are a physicist and respond only in German."),
    HumanMessage(content="Explain quantum mechanics in one sentence.")
]

output = llm.invoke(messages, model="gpt-4o-mini")
print(output.content)

Die Quantenmechanik ist die fundamentale Theorie der Physik, die das Verhalten von Materie und Energie auf subatomarer Ebene beschreibt, wobei sie Phänomene wie Überlagerung, Verschränkung und die Unschärferelation umfasst.


- **SystemMessage**: Represents messages for `System` role
- **AIMessage**: Represents messages for `AI assistant` role
- **HumanMessage**: Represents messages for `User` role

### Caching in Langchain (Caching LLM Responses)

**In memory Cache**

In [9]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [10]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = "Tell me a joke that a toddler can understand."
llm.invoke(prompt)

CPU times: user 287 ms, sys: 23.4 ms, total: 310 ms
Wall time: 1 s


'\n\nWhy did the cookie go to the doctor?\n\nBecause it was feeling crumbly!'

In [11]:
%%time
llm.invoke(prompt)

CPU times: user 460 μs, sys: 62 μs, total: 522 μs
Wall time: 530 μs


'\n\nWhy did the cookie go to the doctor?\n\nBecause it was feeling crumbly!'

**SQLite Caching**

In [12]:
%%time
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))


prompt = "Tell me a joke about America's current presidential elections"
llm.invoke(prompt)

CPU times: user 119 ms, sys: 12.3 ms, total: 132 ms
Wall time: 1.2 s


'\n\nWhy was the election called a circus?\n\nBecause there were so many clowns running for president!'

In [13]:
%%time
llm.invoke(prompt)

CPU times: user 93.6 ms, sys: 41 ms, total: 135 ms
Wall time: 133 ms


'\n\nWhy was the election called a circus?\n\nBecause there were so many clowns running for president!'

**Redis Cache for LangChain**
- See notebook [here](https://python.langchain.com/docs/integrations/caches/redis_llm_caching/)

### Streaming in LangChain (LLM Streaming)

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
prompt = "Write a rock song about the Moon and a Raven."
print(llm.invoke(prompt).content)

Verse 1:
In the dead of night, under the moon's soft glow
A raven takes flight, where no one else would go
With eyes as black as coal, and feathers dark as night
He soars above the world, a creature of eerie might

Chorus:
Moon and raven, dancing in the sky
Silent companions, as the night goes by
With secrets untold, and mysteries to share
They rule the darkness, without a single care

Verse 2:
The moon's pale face, a watchful eye above
Casting shadows in the night, full of mystery and love
The raven calls out, a haunting melody
A song of darkness and light, a symphony of eternity

Chorus:
Moon and raven, dancing in the sky
Silent companions, as the night goes by
With secrets untold, and mysteries to share
They rule the darkness, without a single care

Bridge:
In the stillness of the night, they hold their reign
A timeless bond, they'll never be the same
Together they roam, in the endless night
Moon and raven, a majestic sight

Chorus:
Moon and raven, dancing in the sky
Silent companio

In [15]:
# enable streaming
for chunk in llm.stream(prompt):
    print(chunk.content, end="", flush=True)

Verse 1:
Beneath the pale moonlight
A raven takes to flight
Its wings spread wide
In the still of the night
The moon shines bright
Casting shadows in the sky

Chorus:
Oh, moon and raven
Dancing in the night
Their spirits intertwined
In a cosmic delight
Oh, moon and raven
Guiding us through the dark
With their mysterious spark

Verse 2:
The raven calls out
A haunting melody
Echoing through the trees
In a symphony
The moon watches on
A silent guardian above
A timeless bond
Between earth and sky

Chorus:
Oh, moon and raven
Dancing in the night
Their spirits intertwined
In a cosmic delight
Oh, moon and raven
Guiding us through the dark
With their mysterious spark

Bridge:
They watch over us
In the dead of night
Guiding our way
With their celestial light
Oh, moon and raven
Forever intertwined
In the tapestry of time

Chorus:
Oh, moon and raven
Dancing in the night
Their spirits intertwined
In a cosmic delight
Oh, moon and raven
Guiding us through the dark
With their mysterious spark

Outro:

### Prompt Templates

In [16]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [17]:
template = """Your are an experienced virologist.
Write a few sentences about the following virus {virus} in {language}.
"""

prompt_template = PromptTemplate.from_template(template=template)
prompt = prompt_template.format(virus="ebola", language="english")
print(prompt)

Your are an experienced virologist.
Write a few sentences about the following virus ebola in english.



In [18]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
output = llm.invoke(prompt)
print(output.content)

Ebola virus is a highly pathogenic virus that causes Ebola virus disease (EVD), a severe and often fatal illness in humans and nonhuman primates. It is transmitted through direct contact with bodily fluids of infected individuals or animals, such as fruit bats and primates. Symptoms typically appear 2 to 21 days after exposure and can include fever, severe headache, muscle pain, vomiting, diarrhea, and internal and external bleeding. The virus has a high mortality rate, with some outbreaks reporting fatality rates exceeding 90%. Effective prevention strategies include strict infection control measures, safe burial practices, and vaccination efforts, particularly in outbreak regions.


### ChatPromptTemplates

In [19]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

In [20]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You respond only in the JSON format."),
        HumanMessagePromptTemplate.from_template("Top {n} countries in {area} by population.")
    ]
)

messages = chat_template.format_messages(n="10", area="Europe")
print(messages)

[SystemMessage(content='You respond only in the JSON format.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Top 10 countries in Europe by population.', additional_kwargs={}, response_metadata={})]


In [21]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o-mini")
output = llm.invoke(messages)
print(output.content)

```json
{
  "top_10_countries_in_europe_by_population": [
    {
      "rank": 1,
      "country": "Russia",
      "population": 145912025
    },
    {
      "rank": 2,
      "country": "Germany",
      "population": 83783942
    },
    {
      "rank": 3,
      "country": "United Kingdom",
      "population": 68207114
    },
    {
      "rank": 4,
      "country": "France",
      "population": 65273511
    },
    {
      "rank": 5,
      "country": "Italy",
      "population": 60244639
    },
    {
      "rank": 6,
      "country": "Spain",
      "population": 46754778
    },
    {
      "rank": 7,
      "country": "Poland",
      "population": 38386000
    },
    {
      "rank": 8,
      "country": "Romania",
      "population": 19237691
    },
    {
      "rank": 9,
      "country": "Netherlands",
      "population": 17134872
    },
    {
      "rank": 10,
      "country": "Belgium",
      "population": 11589623
    }
  ]
}
```


In [22]:
messages = chat_template.format_messages(n="10", area="Africa")
output = llm.invoke(messages)
print(output.content)

```json
{
  "top_10_countries_by_population": [
    {
      "rank": 1,
      "country": "Nigeria",
      "population": 223804632
    },
    {
      "rank": 2,
      "country": "Ethiopia",
      "population": 126527200
    },
    {
      "rank": 3,
      "country": "Egypt",
      "population": 109262178
    },
    {
      "rank": 4,
      "country": "Democratic Republic of the Congo",
      "population": 102262000
    },
    {
      "rank": 5,
      "country": "Tanzania",
      "population": 64_185_000
    },
    {
      "rank": 6,
      "country": "South Africa",
      "population": 60_024_000
    },
    {
      "rank": 7,
      "country": "Kenya",
      "population": 56_000_000
    },
    {
      "rank": 8,
      "country": "Uganda",
      "population": 47_300_000
    },
    {
      "rank": 9,
      "country": "Sudan",
      "population": 45_000_000
    },
    {
      "rank": 10,
      "country": "Algeria",
      "population": 44_000_000
    }
  ]
}
```


In [23]:
messages = chat_template.format_messages(n="5", area="Asia")
output = llm.invoke(messages)
print(output.content)

```json
{
  "top_5_countries_by_population": [
    {
      "country": "China",
      "population": 1411778724
    },
    {
      "country": "India",
      "population": 1393409038
    },
    {
      "country": "Indonesia",
      "population": 276361783
    },
    {
      "country": "Pakistan",
      "population": 225199937
    },
    {
      "country": "Bangladesh",
      "population": 166303498
    }
  ]
}
```


In [24]:
messages = chat_template.format_messages(n="5", area="North America")
output = llm.invoke(messages)
print(output.content)

```json
{
  "top_countries_north_america": [
    {
      "country": "United States",
      "population": 331002651
    },
    {
      "country": "Mexico",
      "population": 128932753
    },
    {
      "country": "Canada",
      "population": 37742154
    },
    {
      "country": "Guatemala",
      "population": 17915568
    },
    {
      "country": "Honduras",
      "population": 9904607
    }
  ]
}
```


In [25]:
messages = chat_template.format_messages(n="10", area="World")
output = llm.invoke(messages)
print(output.content)

```json
{
  "top_10_countries_by_population": [
    {
      "rank": 1,
      "country": "China",
      "population": 1411778724
    },
    {
      "rank": 2,
      "country": "India",
      "population": 1393409038
    },
    {
      "rank": 3,
      "country": "United States",
      "population": 331893745
    },
    {
      "rank": 4,
      "country": "Indonesia",
      "population": 276361783
    },
    {
      "rank": 5,
      "country": "Pakistan",
      "population": 225199937
    },
    {
      "rank": 6,
      "country": "Brazil",
      "population": 213993437
    },
    {
      "rank": 7,
      "country": "Nigeria",
      "population": 211400708
    },
    {
      "rank": 8,
      "country": "Bangladesh",
      "population": 166303498
    },
    {
      "rank": 9,
      "country": "Russia",
      "population": 145912025
    },
    {
      "rank": 10,
      "country": "Mexico",
      "population": 130262216
    }
  ]
}
```


### Simple Chains

In [26]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [27]:
llm = ChatOpenAI(model_name="gpt-4o-mini")
template = """Your are an experienced virologist.
Write a few sentences about the following virus {virus} in {language}.
"""
prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

output = chain.invoke({"virus": "covid19", "language": "English"})
print(output)

/tmp/ipykernel_452/4154304358.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


{'virus': 'covid19', 'language': 'English', 'text': 'COVID-19, caused by the novel coronavirus SARS-CoV-2, emerged in late 2019 and rapidly spread worldwide, leading to a global pandemic. The virus primarily transmits through respiratory droplets and can cause a wide range of symptoms, from mild respiratory illness to severe pneumonia and death, particularly in vulnerable populations. Vaccination efforts have been critical in controlling the spread of the virus and reducing the severity of the disease. Ongoing research continues to focus on variants of the virus, their transmissibility, and the effectiveness of vaccines against these variants.'}


#### LangChain Expression Language (LCEL)
- [LCEL](https://python.langchain.com/docs/concepts/lcel/)
- [Build a Simple LLM Application with LCEL](https://python.langchain.com/docs/tutorials/llm_chain/)

In [28]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
prompt = prompt_template
# print(prompt)
chain = prompt | llm
output = chain.invoke({"virus": "COVID-19", "language": "English"})
print(output)

content='COVID-19, caused by the SARS-CoV-2 virus, emerged in late 2019 and rapidly led to a global pandemic. It primarily spreads through respiratory droplets and can cause a wide range of symptoms, from mild respiratory illness to severe pneumonia and death, particularly in vulnerable populations. The virus has the ability to mutate, leading to the emergence of various variants, which can impact transmissibility and vaccine effectiveness. Ongoing research and vaccination efforts are crucial in controlling the spread of COVID-19 and managing its impact on public health.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 107, 'prompt_tokens': 29, 'total_tokens': 136, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None} id='run-0afde1

In [29]:
print(output.content)

COVID-19, caused by the SARS-CoV-2 virus, emerged in late 2019 and rapidly led to a global pandemic. It primarily spreads through respiratory droplets and can cause a wide range of symptoms, from mild respiratory illness to severe pneumonia and death, particularly in vulnerable populations. The virus has the ability to mutate, leading to the emergence of various variants, which can impact transmissibility and vaccine effectiveness. Ongoing research and vaccination efforts are crucial in controlling the spread of COVID-19 and managing its impact on public health.


#### OutputParsers
- [Link](https://python.langchain.com/docs/tutorials/llm_chain/#outputparsers)

In [30]:
chain = prompt | llm | parser
output = chain.invoke({"virus": "COVID-19", "language": "English"})
print(output)

COVID-19, caused by the SARS-CoV-2 virus, emerged in late 2019 and rapidly led to a global pandemic. It primarily spreads through respiratory droplets and can cause a wide range of symptoms, from mild respiratory illness to severe pneumonia and death, particularly in vulnerable populations. The virus has the ability to mutate, leading to the emergence of various variants, which can impact transmissibility and vaccine effectiveness. Ongoing research and vaccination efforts are crucial in controlling the spread of COVID-19 and managing its impact on public health.


In [31]:
template = "What is the capital of {country}?. List the top 3 places to visit in that city. Use bullet points"
prompt_template = PromptTemplate.from_template(template=template)

country = input("Enter Country: ")

# Invoke the chain with specific country
chain = prompt_template | llm | parser
output = chain.invoke(country)
print(output)

Enter Country:  Uganda


The capital of Uganda is Kampala. Here are the top 3 places to visit in the city:

- **Uganda Museum**: This is the oldest museum in East Africa, showcasing the cultural heritage of Uganda with exhibits on archaeology, ethnography, and natural history.

- **Kasubi Tombs**: A UNESCO World Heritage Site, these tombs are the burial grounds for the kings of Buganda and feature traditional architecture and cultural significance.

- **Namirembe Cathedral**: Also known as the Cathedral of the Diocese of Namirembe, this Anglican cathedral is known for its stunning architecture and panoramic views of Kampala.


In [32]:
from IPython.display import Markdown, display

# Display the output as markdown
display(Markdown(output))

The capital of Uganda is Kampala. Here are the top 3 places to visit in the city:

- **Uganda Museum**: This is the oldest museum in East Africa, showcasing the cultural heritage of Uganda with exhibits on archaeology, ethnography, and natural history.

- **Kasubi Tombs**: A UNESCO World Heritage Site, these tombs are the burial grounds for the kings of Buganda and feature traditional architecture and cultural significance.

- **Namirembe Cathedral**: Also known as the Cathedral of the Diocese of Namirembe, this Anglican cathedral is known for its stunning architecture and panoramic views of Kampala.

### Sequential Chains

Output of one chain can be used as the input of another chain

In [33]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Initialize the first ChatOpenAI model (gpt-4o-mini) with specific temperature
llm1 = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.5)

# Define the first prompt template
prompt_template1 = PromptTemplate.from_template(
    template='You are an experienced scientist and Python programmer. Write a function that implements the concept of {concept}.'
)
# Create an LLMChain using the first model and the prompt template
chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

# Initialize the second ChatOpenAI model (gpt-4o) with specific temperature
llm2 = ChatOpenAI(model_name='gpt-4o', temperature=1.2)

# Define the second prompt template
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)
# Create another LLMChain using the second model and the prompt template
chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

# Combine both chains into a SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

# Invoke the overall chain with the concept "linear regression"
output = overall_chain.invoke('linear regression')



> Entering new SimpleSequentialChain chain...
Certainly! Below is a Python function that implements simple linear regression using the least squares method. This function will take in two lists or arrays: one for the independent variable \( x \) and another for the dependent variable \( y \). It will return the slope and intercept of the best-fit line, as well as the predicted values.

We'll use NumPy for numerical operations. If you don't have NumPy installed, you can install it using `pip install numpy`.

Here’s the implementation:

```python
import numpy as np

def linear_regression(x, y):
    """
    Perform simple linear regression on the given data.

    Parameters:
    x (array-like): Independent variable data.
    y (array-like): Dependent variable data.

    Returns:
    slope (float): The slope of the regression line.
    intercept (float): The y-intercept of the regression line.
    predictions (numpy array): Predicted values for y based on the regression line.
    """
   

In [34]:
display(Markdown(output["output"]))

Certainly! The Python function provided implements simple linear regression using the least squares method, which is a statistical technique to model the relationship between a dependent variable \( y \) and an independent variable \( x \). Linear regression aims to find the best-fit line through the data points to minimize the difference between observed and predicted values.

### Detailed Explanation of the Python Function:

1. **Importing NumPy**:
   - The function makes use of NumPy, a popular library for numerical operations in Python, for efficient computation.

2. **Function Definition**:  
   - The function `linear_regression(x, y)` accepts two input parameters:
     - `x` (array-like): Represents the independent variable data, the predictor.
     - `y` (array-like): Represents the dependent variable data, which the function attempts to predict.

3. **Conversion to NumPy Arrays**:
   - The inputs `x` and `y` are converted into NumPy arrays to leverage NumPy’s mathematical and statistical functions.

4. **Mean Calculation**:
   - `x_mean = np.mean(x):` Computes the average of the `x` values.
   - `y_mean = np.mean(y):` Computes the average of the `y` values.

5. **Calculating the Slope (\( m \))**:
   - The numerator for the slope is computed as the sum of the product of the differences from their respective means:
     \[
     \text{numerator} = \sum{(x_i - \bar{x})(y_i - \bar{y})}
     \]
   - The denominator is the sum of squared differences from the mean of \( x \):
     \[
     \text{denominator} = \sum{(x_i - \bar{x})^2}
     \]
   - The slope is then calculated as:
     \[
     m = \frac{\text{numerator}}{\text{denominator}}
     \]

6. **Calculating the Intercept (\( b \))**:
   - The intercept is derived from:
     \[
     b = \bar{y} - m \cdot \bar{x}
     \]
   - This formula adjusts the level of the regression line to better fit the data.

7. **Making Predictions**:
   - For each input value of \( x \), predictions are made with the linear equation of the line:
     \[
     \text{predictions} = m \cdot x + b
     \]

8. **Return Values**:
   - The function returns three outputs: the slope, intercept, and the array of predicted \( y \) values based on the input \( x \).

### Example Usage:
An example demonstrates using the function where \( x \) and \( y \) represent sample data - in this case, a simple sequence:
   - \( x \) values: \([1, 2, 3, 4, 5]\)
   - \( y \) values: \([2, 3, 5, 7, 11]\)

The output results indicate:
- **Slope**: Indicates the rate at which the dependent variable \( y \) changes per unit change in \( x \).
- **Intercept**: Represents the expected mean value when \( x \) is zero.
- **Predictions**: Estimated dependent variable values when using the regression line equation.

### Application:
This linear regression function can be applied to numerous areas, including economics, biology, engineering, etc., for exploratory data analysis, prediction, and inferring relationships between variables where a linear relationship is expected. The simplicity of the model adds the advantage of quick computation and easy interpretability.

By replacing the sample data with any dataset of interest, the user can compute slope, intercept, and generate predictions for understanding patterns or forecasting future data points.

#### Using LangChain Expression Language (LCEL) for Sequential chains

In [35]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Initialize the first ChatOpenAI model (gpt-4o-mini) with specific temperature
llm1 = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.5)

# Define the first prompt template
prompt_template1 = PromptTemplate.from_template(
    template='You are an experienced scientist and Python programmer. Write a function that implements the concept of {concept}.'
)

# Initialize the second ChatOpenAI model (gpt-4o) with specific temperature
llm2 = ChatOpenAI(model_name='gpt-4o', temperature=1.2)

# Define the second prompt template
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)

# The output parser will handle string output for both chains
parser = StrOutputParser()

# Use the pipe operator (|) to create the sequence of steps using LCEL
chain = (
    prompt_template1 
    | llm1 
    | prompt_template2 
    | llm2 
    | parser
)

# Invoke the chain with the input 'linear regression'
output = chain.invoke({'concept': 'linear regression'})

# Print the result
print(output)


Certainly! Below is a thorough explanation of a Python function that implements simple linear regression through the least squares method. This function specifically leverages NumPy for computational efficiency and clarity. It is designed to take two lists or arrays as inputs: one representing the independent variable \( x \) and the other the dependent variable \( y \). The function returns the calculated slope and intercept of the best-fit line and also provides the predicted values for \( y \) based on this line. Let’s go through the function step-by-step, and provide further insights regarding its components.

### Code Overview

The function is defined as:

```python
import numpy as np

def linear_regression(x, y):
    """
    Perform simple linear regression on the given data.

    Parameters:
    x (array-like): Independent variable data.
    y (array-like): Dependent variable data.

    Returns:
    slope (float): The slope of the regression line.
    intercept (float): The y-in

In [36]:
display(Markdown(output))

Certainly! Below is a thorough explanation of a Python function that implements simple linear regression through the least squares method. This function specifically leverages NumPy for computational efficiency and clarity. It is designed to take two lists or arrays as inputs: one representing the independent variable \( x \) and the other the dependent variable \( y \). The function returns the calculated slope and intercept of the best-fit line and also provides the predicted values for \( y \) based on this line. Let’s go through the function step-by-step, and provide further insights regarding its components.

### Code Overview

The function is defined as:

```python
import numpy as np

def linear_regression(x, y):
    """
    Perform simple linear regression on the given data.

    Parameters:
    x (array-like): Independent variable data.
    y (array-like): Dependent variable data.

    Returns:
    slope (float): The slope of the regression line.
    intercept (float): The y-intercept of the regression line.
    predictions (numpy array): Predicted values for y based on the regression line.
    """
    # Convert input to numpy arrays
    x = np.array(x)
    y = np.array(y)

    # Calculate the means of x and y
    x_mean = np.mean(x)
    y_mean = np.mean(y)

    # Calculate the slope (m)
    numerator = np.sum((x - x_mean) * (y - y_mean))
    denominator = np.sum((x - x_mean)**2)
    slope = numerator / denominator

    # Calculate the intercept (b)
    intercept = y_mean - (slope * x_mean)

    # Calculate predicted values
    predictions = slope * x + intercept

    return slope, intercept, predictions
```

### Detailed Explanation

#### 1. **Imports**

- **NumPy**: The function uses NumPy, a library renowned for handling arrays and matrices efficiently and providing mathematical functions. Importing NumPy as `np` is a conventional practice in Python, providing a shorthand usage throughout the code.

#### 2. **Function Definition and Docstring**

- **Docstring**: The `linear_regression` function begins with a detailed docstring stating its purpose, input parameters, and the values it returns:
  - **Parameters**: 
    - `x`: Represents the array-like structure of the independent variable.
    - `y`: Represents the dependent variable data which the function aims to model using a linear relationship.
  - **Returns**:
    - `slope`: The line's slope indicating the relationship strength between two variables.
    - `intercept`: Determines where the regression line intercepts the y-axis.
    - `predictions`: The y-values predicted by the regression model.

#### 3. **Data Conversion**

- **To NumPy arrays**: The function starts by converting the input lists or arrays into NumPy arrays for efficient operations (`x = np.array(x)` and `y = np.array(y)`). This transformation enables the use of rich NumPy functionalities such as vectorized operations and built-in methods for calculations.

#### 4. **Mean Calculation**

- **Mean of x and y**: It calculates means using `np.mean(x)` and `np.mean(y)`. These means are crucial for computing the slope and the intercept.
  
#### 5. **Slope Calculation**

- **Formula**: The slope is determined using the formula:
  \[
  m = \frac{\sum{(x_i - \bar{x})(y_i - \bar{y})}}{\sum{(x_i - \bar{x})^2}}
  \]
- This involves two main parts:
  - **Numerator**: It's the sum of products of variable differences from their mean.
  - **Denominator**: The sum of squares of x-differences from its mean. 
- **Calculation**: `slope = numerator / denominator`

#### 6. **Intercept Calculation**

- **Formula**: Using the calculated slope, the intercept is given by:
  \[
  b = \bar{y} - m \cdot \bar{x}
  \]
- **Application**: Automatically shifts the best-fit line to align with observed data points over the x-y plane.

#### 7. **Prediction Calculation**

- **Predicted y-values**: Using the equation \( y = mx + b \), the function calculates predictions portraying the regression line.

#### 8. **Return Statement**

- Returns the slope, intercept, and calculated predictions, summing the function’s utility as a practical regression implementation tool.

### Example Usage

```python
# Example usage
if __name__ == "__main__":
    # Sample data
    x = [1, 2, 3, 4, 5]
    y = [2, 3, 5, 7, 11]

    slope, intercept, predictions = linear_regression(x, y)
    print(f"Slope: {slope}")
    print(f"Intercept: {intercept}")
    print(f"Predicted values: {predictions}")
```

Here, the script explores a sample dataset and executes the regression function, yielding the slope, intercept, and predicted values for displayed simple progression in data point sequences: translating the model read into numerical outcomes defining linearity.

### Conclusion

The provided linear regression function is a fundamental tool for establishing relationships between datasets whereby dependent response is envisioned versatilely influenced by a singular predictor value in isolated scenarios. Utilizing efficient computational resources, this method swiftly reveals statistically inferred trends necessary for many analytical pursuits.

### LangChain Agents

#### Langchain Agents in Action: Python REPL
- [How to: use legacy LangChain Agents (AgentExecutor)](https://python.langchain.com/docs/how_to/agent_executor/)

In [37]:
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
python_repl.run("print([n for n in range(1, 100) if n % 3 == 0])")

Python REPL can execute arbitrary code. Use with caution.


'[3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99]\n'

In [38]:
python_repl.run("print([n for n in range(1, 100) if n % 13 == 0])")

'[13, 26, 39, 52, 65, 78, 91]\n'

In [39]:
# Using langchain agents
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

# Invoke the agent
prompt = 'Calculate the square root of the factorial of 12 and display it with 4 decimal points'
agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...
To calculate the square root of the factorial of 12, I will first compute the factorial of 12, then take the square root of that result. Finally, I will format the output to display it with 4 decimal points. 

Action: Python_REPL  
Action Input: `import math; result = math.sqrt(math.factorial(12)); print(f"{result:.4f}")`  
Observation: 21886.1052

Thought:I now know the final answer  
Final Answer: 21886.1052

> Finished chain.


{'input': 'Calculate the square root of the factorial of 12 and display it with 4 decimal points',
 'output': '21886.1052'}

In [66]:
prompt = "What is the answer to 5.1 ** 7.3?"
agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...
To find the answer to \(5.1^{7.3}\), I need to calculate the power of 5.1 raised to 7.3 using Python. 

Action: Python_REPL  
Action Input: `5.1 ** 7.3`  
Observation: 
Thought:I need to print the result of the calculation to see the output.  
Action: Python_REPL  
Action Input: `print(5.1 ** 7.3)`  
Observation: 146306.05007233328

Thought:I now know the final answer  
Final Answer: 146306.05007233328

> Finished chain.


{'input': 'What is the answer to 5.1 ** 7.3?', 'output': '146306.05007233328'}

#### How to migrate from legacy LangChain agents to LangGraph
- [LangGraph](https://python.langchain.com/docs/how_to/migrate_agent/)
- [How to: migrate from legacy LangChain agents to LangGraph](https://python.langchain.com/docs/how_to/migrate_agent/)
- [LangGraph Docs](https://langchain-ai.github.io/langgraph/)
- [Tool calling](https://python.langchain.com/docs/how_to/tool_calling/)

In [42]:
%%capture
!pip install -qU langgraph

In [47]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_experimental.tools.python.tool import PythonREPLTool

In [60]:
# help(create_react_agent)

In [61]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
tools = [PythonREPLTool()]
langgraph_agent_executor = create_react_agent(llm, tools)

prompt = 'Calculate the square root of the factorial of 12 and display it with 4 decimal points'
messages = langgraph_agent_executor.invoke({"messages": [("human", prompt)]})

In [62]:
messages

{'messages': [HumanMessage(content='Calculate the square root of the factorial of 12 and display it with 4 decimal points', additional_kwargs={}, response_metadata={}, id='e87010a0-1a02-46c5-b64c-444aef92d565'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_fXg8ptZeq8JF4k8MkwK4kEOn', 'function': {'arguments': '{"query":"import math\\nresult = math.sqrt(math.factorial(12))\\nprint(f\'{result:.4f}\')"}', 'name': 'Python_REPL'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 96, 'total_tokens': 134, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-292181fc-2f3b-462a-9dd9-734a9d30d597-0', tool_calls=[{'name': 'Python_REPL', 'args': {'query': "import math\nresult = ma

In [63]:
print(messages["messages"][-1].content)

The square root of the factorial of 12 is approximately 21886.1052.


In [78]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
langgraph_agent_executor = create_react_agent(llm, tools)
prompt = "What is the answer to 5.1 ** 7.3?"

messages = langgraph_agent_executor.invoke({"messages": [("human", prompt)]})

In [79]:
messages

{'messages': [HumanMessage(content='What is the answer to 5.1 ** 7.3?', additional_kwargs={}, response_metadata={}, id='f72a1901-cca3-46bf-9d28-d06ce20e851d'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hdr1ybL4rohbJZRBz8eIcmPI', 'function': {'arguments': '{"query":"result = 5.1 ** 7.3\\nprint(result)"}', 'name': 'Python_REPL'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 93, 'total_tokens': 123, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_159d8341cc', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-87644859-371b-415c-8be6-d7683812ac3e-0', tool_calls=[{'name': 'Python_REPL', 'args': {'query': 'result = 5.1 ** 7.3\nprint(result)'}, 'id': 'call_hdr1ybL4rohbJZRBz8eIcmPI', 'type': 'tool_call'}], usage_metadata={'input_to

In [80]:
print(messages["messages"][-1].content)

The answer to \(5.1^{7.3}\) is approximately 146,306.05.


In [77]:
5.1 ** 7.3

146306.05007233328

### Langchain Tools: DuckDuckGo, Wikipedia and [tavily AI](https://tavily.com/)
#### DuckDuckGo Search

In [81]:
!pip install -q duckduckgo-search

In [82]:
# optional, filter workings (recommended in production)
import warnings
warnings.filterwarnings('ignore', module='langchain')

In [83]:
from langchain.tools import DuckDuckGoSearchRun

In [84]:
search = DuckDuckGoSearchRun()
output = search.invoke("who is Leonel Messi?")
display(Markdown(output))

Lionel Messi (born June 24, 1987, Rosario, Argentina) is an Argentine-born football (soccer) player who received a record-setting eight Ballon d'Or awards as the world's top male player (2009-12, 2015, 2019, 2021, and 2023). In 2022 he helped Argentina win the Fédération Internationale de Football Association (FIFA)'s World Cup.. Early life. Messi started playing football as a boy ... Lionel Messi is one of the world's best soccer players and helped Argentina win the 2022 FIFA World Cup. Read about his teams, awards, wife, height, and more. Messi's full name is Lionel Andrés Messi. Lionel Andrés Messi, widely known as Messi, was born on June 24, 1987, in Rosario, Argentina. His exceptional football skills have earned him global recognition and numerous accolades throughout his career. Messi joined FC Barcelona at the age of 13. An exec for Messi's longtime sponsor Adidas helped broker a meeting with Beckham, Jorge Mas, and Jorge Messi, Lionel's father and agent, in September 2019 in Barcelona. Lionel Messi is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team.His individual achievements include eight Ballon d'Or awards, the most for any footballer. Having won 45 team trophies, [note 1] he is the most decorated player in the history of professional football. [11]

In [85]:
search.name

'duckduckgo_search'

In [86]:
search.description

'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.'

#### To get more details

In [87]:
from langchain.tools import DuckDuckGoSearchResults

In [88]:
search = DuckDuckGoSearchResults()
output = search.run("How many Ballon d'Or awards did Lionel Messi win?")
display(Markdown(output))

snippet: Alfredo Di Stéfano. Real Madrid. Argentina/Spain. 1956. Stanley Matthews. Blackpool. England. Lionel Messi and Cristiano Ronaldo headline the historic list of Ballon d'Or winners., title: Men's Ballon d'Or: Full List of Winners - Sports Illustrated, link: https://www.si.com/soccer/men-s-ballon-d-or-full-list-of-winners, snippet: Messi and Ronaldo shared the Ballon d'Or between themselves from 2008-2017 Credit: TalkSPORT. France Football's top prize became synonymous with the epic Lionel Messi-Cristiano Ronaldo rivalry ..., title: Ballon d'Or winners in full: Lionel Messi, Cristiano Ronaldo and every ..., link: https://talksport.com/football/2204133/ballon-dor-winners-1956-lionel-messi-cristiano-ronaldo/, snippet: Lionel Messi's Ballon d'Or wins. After finishing third in 2007 and second in 2008, Messi won his first Ballon d'Or in 2009, before going on to win the award for the following three successive years., title: How many times has Lionel Messi won the Ballon d'Or? Past trophy wins ..., link: https://www.sportingnews.com/us/soccer/news/how-many-times-lionel-messi-won-ballon-dor-argentina/ed1ab61e49877c49d2397c38, snippet: 2019: Lionel Messi (Barcelona) 2020: Cancelled because of Covid-19; 2021: Lionel Messi (Paris Saint-Germain) 2022: Karim Benzema (Real Madrid) 2023: Lionel Messi (Inter Miami) 2024: Rodri (Man ..., title: The full list of Ballon d'Or winners - from 1956 to the present day, link: https://inews.co.uk/sport/football/ballon-dor-winners-full-list-history-206332

#### To take more control of the search

In [89]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

In [91]:
# help(DuckDuckGoSearchAPIWrapper)

In [96]:
wrapper = DuckDuckGoSearchAPIWrapper(region='us-en', max_results=3, safesearch='moderate')
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source='politics')
output = search.run('Current us president')

In [93]:
output

'snippet: Updated results from the 2024 election for the US president. Reuters live coverage of the 2024 US President, Senate, House and state governors races., title: US presidential election results 2024: Harris vs. Trump | Live maps ..., link: https://www.reuters.com/graphics/USA-ELECTION/RESULTS/zjpqnemxwvx/, snippet: O n November 5th America will pick its next president, as well as the House of Representatives and a third of the Senate. The Economist will publish live results and analysis covering every race., title: US Presidential election: live results | The Economist, link: https://www.economist.com/interactive/us-2024-election/results/president, snippet: Historical election results for these districts are calculated based on votes cast within the current boundaries of the district. Sources: Polling averages by The New York Times. Individual polls ..., title: Election 2024 Polls: Harris vs. Trump - The New York Times, link: https://www.nytimes.com/interactive/2024/us/elections

In [101]:
import re

# Regular expression to extract snippet, title, and link
pattern = r"snippet: (.*?), title: (.*?), link: (https?://\S+)"

# Find all matches
matches = re.findall(pattern, output)

# Display the extracted matches
matches


[('Updated results from the 2024 election for the US president. Reuters live coverage of the 2024 US President, Senate, House and state governors races.',
  'US presidential election results 2024: Harris vs. Trump | Live maps ...',
  'https://www.reuters.com/graphics/USA-ELECTION/RESULTS/zjpqnemxwvx/,'),
 ('This web page provides a comprehensive list of all the presidents of the United States from 1789 to the present, with their portraits, terms, parties, and vice presidents. The current president is Joe Biden, who assumed office on January 20, 2021.',
  'List of presidents of the United States - Wikipedia',
  'https://en.wikipedia.org/wiki/List_of_Presidents_of_the_United_States,'),
 ('O n November 5th America will pick its next president, as well as the House of Representatives and a third of the Senate. The Economist will publish live results and analysis covering every race.',
  'US Presidential election: live results - The Economist',
  'https://www.economist.com/interactive/us-20

In [102]:
for snippet, title, link in matches:
    print(f'Snippet: {snippet}\nTitle: {title}\nLink: {link}\n')
    print('-' * 50)

Snippet: Updated results from the 2024 election for the US president. Reuters live coverage of the 2024 US President, Senate, House and state governors races.
Title: US presidential election results 2024: Harris vs. Trump | Live maps ...
Link: https://www.reuters.com/graphics/USA-ELECTION/RESULTS/zjpqnemxwvx/,

--------------------------------------------------
Snippet: This web page provides a comprehensive list of all the presidents of the United States from 1789 to the present, with their portraits, terms, parties, and vice presidents. The current president is Joe Biden, who assumed office on January 20, 2021.
Title: List of presidents of the United States - Wikipedia
Link: https://en.wikipedia.org/wiki/List_of_Presidents_of_the_United_States,

--------------------------------------------------
Snippet: O n November 5th America will pick its next president, as well as the House of Representatives and a third of the Senate. The Economist will publish live results and analysis coverin

#### Wikipidea

In [103]:
%%capture
!pip install -q wikipedia

In [112]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
warnings.filterwarnings('ignore')

In [106]:
api_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=10000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
output = wiki.invoke({'query': 'llamaindex'})
display(Markdown(output))

Page: NebulaGraph
Summary: NebulaGraph is an open-source distributed graph database built for super large-scale graphs with milliseconds of latency. NebulaGraph adopts the Apache 2.0 license and  also  comes with a wide range of data visualization tools.

Page: Vector database
Summary: A vector database, vector store or vector search engine is a database that can store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more Approximate Nearest Neighbor algorithms, so that one can search the database with a query vector to retrieve the closest matching database records.
Vectors are mathematical representations of data in a high-dimensional space. In this space, each dimension corresponds to a feature of the data, with the number of dimensions ranging from a few hundred to tens of thousands, depending on the complexity of the data being represented. A vector's position in this space represents its characteristics. Words, phrases, or entire documents, as well as images, audio, and other types of data, can all be vectorized.
These feature vectors may be computed from the raw data using machine learning methods such as feature extraction algorithms, word embeddings or deep learning networks. The goal is that semantically similar data items receive feature vectors close to each other.
Vector databases can be used for similarity search, semantic search, multi-modal search, recommendations engines, large language models (LLMs), object detection,  etc.
Vector databases are also often used to implement retrieval-augmented generation (RAG), a method to improve domain-specific responses of large language models. The retrieval component of a RAG can be any search system, but is most often implemented as a vector database. Text documents describing the domain of interest are collected, and for each document or document section, a feature vector (known as an "embedding") is computed, typically using a deep learning network, and stored in a vector database. Given a user prompt, the feature vector of the prompt is computed, and the database is queried to retrieve the most relevant documents. These are then automatically added into the context window of the large language model, and the large language model proceeds to create a response to the prompt given this context.

Page: Prompt engineering
Summary: Prompt engineering is the process of structuring an instruction that can be interpreted and understood by a generative artificial intelligence (AI) model. 
A prompt is natural language text describing the task that an AI should perform. A prompt for a text-to-text language model can be a query such as "what is Fermat's little theorem?", a command such as "write a poem in the style of Edgar Allan Poe about leaves falling", or a longer statement including context, instructions, and conversation history.
Prompt engineering may involve phrasing a query, specifying a style, providing relevant context or assigning a role to the AI such as "act as a native French speaker".
When communicating with a text-to-image or a text-to-audio model, a typical prompt is a description of a desired output such as "a high-quality photo of an astronaut riding a horse" or "Lo-fi slow BPM electro chill with organic samples". Prompting a text-to-image model may involve adding, removing, emphasizing, and re-ordering words to achieve a desired subject, style, layout, lighting, and aesthetic.

In [113]:
output = wiki.invoke("Google Gemini")
display(Markdown(output))

Page: Gemini (chatbot)
Summary: Gemini, formerly known as Bard, is a generative artificial intelligence chatbot developed by Google. Based on the large language model (LLM) of the same name, it was launched in 2023 after being developed as a direct response to the rise of OpenAI's ChatGPT. It was previously based on PaLM, and initially the LaMDA family of large language models.
LaMDA had been developed and announced in 2021, but it was not released to the public out of an abundance of caution. OpenAI's launch of ChatGPT in November 2022 and its subsequent popularity caught Google executives off-guard, prompting a sweeping response in the ensuing months. After mobilizing its workforce, the company launched Bard in a limited capacity in March 2023 before expanding to other countries in May. Bard took center stage during the 2023 Google I/O keynote in May and was upgraded to the Gemini LLM in December. In February 2024, Bard and Duet AI, another artificial intelligence product from Google, were unified under the Gemini brand, coinciding with the launch of an Android app.

Page: Gemini (language model)
Summary: Gemini is a family of multimodal large language models developed by Google DeepMind, serving as the successor to LaMDA and PaLM 2. Comprising Gemini Ultra, Gemini Pro, Gemini Flash, and Gemini Nano, it was announced on December 6, 2023, positioned as a competitor to OpenAI's GPT-4. It powers the chatbot of the same name.



## ReAct (Reasioning and Acting) Agent
### Create ReAct Agent

In [114]:
%%capture
!pip install langchainhub -q

In [115]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

In [116]:
# Initialize the ChatOpenAI model (gpt-4-turbo-preview) with a temperature of 0. Utilize gpt-3.5-turbo if you use the free plan
llm = ChatOpenAI(model_name='gpt-4o', temperature=0)

# Define a template for answering questions
template = '''
Answer the following questions in ENGLISH as best you can.
Questions: {q}
'''

# Create a PromptTemplate object from the template
prompt_template = PromptTemplate.from_template(template)

# Pull the react prompt from the hub
prompt = hub.pull('hwchase17/react')

# displaying information about the react prompt
# print(type(prompt))
# print(prompt.input_variables)
# print(prompt.template)


# Create tools for the agent

# 1. Python REPL Tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Useful when you need to use Python to answer a question. You should input Python code.'
)

# 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on Wikipedia.'
)

# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool can\'t provide.'
)

# Combine the tools into a list
tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

# Create a react agent with the ChatOpenAI model, tools, and prompt
agent = create_react_agent(llm, tools, prompt)

# Initialize the AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

In [117]:
question = 'Generate the first 20 numbers in the Fibonacci series.'

output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
To generate the first 20 numbers in the Fibonacci series, I can use a simple Python script. The Fibonacci series is a sequence where each number is the sum of the two preceding ones, usually starting with 0 and 1.

Action: Python REPL
Action Input: 
```python
def fibonacci_series(n):
    fib_series = [0, 1]
    while len(fib_series) < n:
        fib_series.append(fib_series[-1] + fib_series[-2])
    return fib_series

fibonacci_series(20)
```
Observation: The first 20 numbers in the Fibonacci series are:

\[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181\]

Final Answer: The first 20 numbers in the Fibonacci series are: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181.

> Finished chain.


In [119]:
print(output["output"])

The first 20 numbers in the Fibonacci series are: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181.


In [122]:
question = 'Who is the current prime minister of the UK as of 2024?'
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
To find out who the current Prime Minister of the UK is as of 2024, I should perform an internet search to get the most up-to-date information.

Action: DuckDuckGo Search
Action Input: "current Prime Minister of the UK 2024"Sir Keir Starmer is the new prime minister of the United Kingdom. The 2024 general election saw the Conservative Party lose power for the first time since 2010. So who is Sir Keir Starmer? Labour has won the UK general election in a landslide. The new Prime Minister, Keir Starmer, pledged to steer the country towards "calmer waters" in his first address to the nation. Starmer is ... Keir Starmer, a former lawyer and Labour Party leader, became the U.K. prime minister in 2024 after his party won a historic election victory. Learn about his background, policies and challenges ... Prime Minister Keir Starmer took office in Britain on Friday and promised a "national renewal" after his center-left Labour Party won a landslide elect

In [123]:
print(output["output"])

As of 2024, the current Prime Minister of the United Kingdom is Keir Starmer.


In [124]:
question = 'Tell me about Kamala Harris early life'
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
To provide a comprehensive answer about Kamala Harris's early life, I should look up detailed information. Wikipedia is a reliable source for biographical information. 

Action: Wikipedia
Action Input: Kamala Harris early lifePage: Early life and career of Kamala Harris
Summary: Kamala Devi Harris was born in Oakland, California, 1964 to biologist Shyamala Gopalan and economist Donald J. Harris. The Harris family moved to various locations in the Midwestern United States from 1966 to 1970, when she moved back to California. At the age of twelve, she moved to Montreal, Quebec, where she attended school through her first year of college.  She then attended Howard University and the University of California College of the Law, San Francisco.
In her early career, Harris served as Alameda County Deputy District Attorney, then San Francisco County Assistant District Attorney, and then running the Family and Children's Services Division in the San Franc

In [125]:
print(output["output"])

Kamala Harris was born on October 20, 1964, in Oakland, California, to Shyamala Gopalan, a biologist from India, and Donald J. Harris, an economist from Jamaica. Her family moved to various locations in the Midwestern United States from 1966 to 1970, before returning to California. At the age of twelve, Kamala moved to Montreal, Quebec, where she attended school through her first year of college. She later attended Howard University in Washington, D.C., and the University of California College of the Law, San Francisco.

In her early career, Kamala Harris served as an Alameda County Deputy District Attorney, then as a San Francisco County Assistant District Attorney, and later ran the Family and Children's Services Division in the San Francisco City Attorney's Office. She served two terms as the District Attorney of San Francisco from 2004 to 2011.
